In [7]:
import yfinance as yf
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

In [2]:
# Get all the S&P500 tickers
path = './Data/S&P500/'
dir_path = [name[:-4] for name in os.listdir(path)]
print(dir_path)

['CSCO', 'UAL', 'TROW', 'ISRG', 'NVR', 'TPR', 'DVN', 'CE', 'MRO', 'BA', 'VRTX', 'GILD', 'NLSN', 'EQIX', 'TER', 'MDT', 'V', 'QRVO', 'A', 'FOX', 'FLT', 'MO', 'CTRA', 'SWKS', 'ENPH', 'MCHP', 'CDNS', 'MSCI', 'CHTR', 'EIX', 'KDP', 'BBY', 'WBA', 'LVS', 'HCA', 'AJG', 'DTE', 'C', 'T', 'CF', 'DISH', 'MGM', 'HUM', 'CBOE', 'CFG', 'APH', 'SYY', 'MSI', 'FCX', 'ADM', 'OGN', 'LH', 'PKI', 'LNT', 'BAC', 'LNC', 'PSX', 'GPN', 'PPG', 'TECH', 'IRM', 'IQV', 'ESS', 'WBD', 'HAL', 'STZ', 'DXC', 'PARA', 'ADI', 'F', 'ADBE', 'CPRT', 'TDG', 'TFX', 'ULTA', 'ARE', 'SYK', 'CB', 'TSN', 'GNRC', 'PEP', 'PEG', 'NOW', 'LLY', 'COST', 'REG', 'NWS', 'LOW', 'MDLZ', 'BKNG', 'ZBRA', 'FMC', 'XEL', 'AIZ', 'MET', 'FTV', 'DLR', 'XRAY', 'FAST', 'TJX', 'SNA', 'MPC', 'BR', 'D', 'MRK', 'STX', 'NOC', 'BXP', 'KHC', 'IPG', 'UNP', 'ALLE', 'ABBV', 'CDAY', 'ORCL', 'ECL', 'ETR', 'EBAY', 'SBUX', 'PENN', 'IR', 'AMT', 'INTU', 'DPZ', 'PAYC', 'DRE', 'CMA', 'PG', 'CAT', 'ODFL', 'MCD', 'MNST', 'AMZN', 'INTC', 'PNR', 'GLW', 'BDX', 'KMI', 'PWR', 'APTV

In [3]:
print(len(dir_path))

503


In [10]:
# Get the history price MAX
GetInfo = yf.Ticker("AAPL")

data = GetInfo.history(period="max").reset_index()

data

                           Date        Open        High         Low  \
0     1980-12-12 00:00:00-05:00    0.099319    0.099750    0.099319   
1     1980-12-15 00:00:00-05:00    0.094569    0.094569    0.094137   
2     1980-12-16 00:00:00-05:00    0.087659    0.087659    0.087228   
3     1980-12-17 00:00:00-05:00    0.089387    0.089818    0.089387   
4     1980-12-18 00:00:00-05:00    0.091978    0.092410    0.091978   
...                         ...         ...         ...         ...   
10872 2024-01-30 00:00:00-05:00  190.940002  191.800003  187.470001   
10873 2024-01-31 00:00:00-05:00  187.039993  187.100006  184.350006   
10874 2024-02-01 00:00:00-05:00  183.990005  186.949997  183.820007   
10875 2024-02-02 00:00:00-05:00  179.860001  187.330002  179.250000   
10876 2024-02-05 00:00:00-05:00  188.149994  189.250000  185.839996   

            Close     Volume  Dividends  Stock Splits  
0        0.099319  469033600        0.0           0.0  
1        0.094137  175884800       

/Users/sabrinachiang/miniforge3/envs/StockPrice/lib/python3.10/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


In [12]:
data = data[['Date', 'Close']]
data

,Date,Close
0,1980-12-12 00:00:00-05:00,0.099319
1,1980-12-15 00:00:00-05:00,0.094137
2,1980-12-16 00:00:00-05:00,0.087228
3,1980-12-17 00:00:00-05:00,0.089387
4,1980-12-18 00:00:00-05:00,0.091978
...,...,...
10872,2024-01-30 00:00:00-05:00,188.039993
10873,2024-01-31 00:00:00-05:00,184.399994
10874,2024-02-01 00:00:00-05:00,186.860001
10875,2024-02-02 00:00:00-05:00,185.850006


In [13]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [20]:
# Generate the 
from copy import deepcopy as dc

def prepare_data(df, nsteps):
    '''
    for each day, we append n previous days 
    '''
    df = dc(df)
    df.set_index('Date', inplace = True)
    
    for i in range(1, nsteps):
        df[f'Close(t-{i})D'] = df['Close'].shift(i)
        
    df.dropna(inplace= True)
    
    return df

In [21]:
lookback = 7
shift_df = prepare_data(data, lookback)

shift_df

,Close,Close(t-1)D,Close(t-2)D,Close(t-3)D,Close(t-4)D,Close(t-5)D,Close(t-6)D
Date,,,,,,,
1980-12-22 00:00:00-05:00,0.102341,0.097591,0.091978,0.089387,0.087228,0.094137,0.099319
1980-12-23 00:00:00-05:00,0.106660,0.102341,0.097591,0.091978,0.089387,0.087228,0.094137
1980-12-24 00:00:00-05:00,0.112273,0.106660,0.102341,0.097591,0.091978,0.089387,0.087228
1980-12-26 00:00:00-05:00,0.122637,0.112273,0.106660,0.102341,0.097591,0.091978,0.089387
1980-12-29 00:00:00-05:00,0.124364,0.122637,0.112273,0.106660,0.102341,0.097591,0.091978
...,...,...,...,...,...,...,...
2024-01-30 00:00:00-05:00,188.039993,191.729996,192.419998,194.169998,194.500000,195.179993,193.889999
2024-01-31 00:00:00-05:00,184.399994,188.039993,191.729996,192.419998,194.169998,194.500000,195.179993
2024-02-01 00:00:00-05:00,186.860001,184.399994,188.039993,191.729996,192.419998,194.169998,194.500000


In [22]:
shift_numpy = shift_df.to_numpy()
shift_numpy

array([[1.02341257e-01, 9.75914821e-02, 9.19781625e-02, ...,
        8.72276649e-02, 9.41371247e-02, 9.93186459e-02],
       [1.06659941e-01, 1.02341257e-01, 9.75914821e-02, ...,
        8.93866271e-02, 8.72276649e-02, 9.41371247e-02],
       [1.12273216e-01, 1.06659941e-01, 1.02341257e-01, ...,
        9.19781625e-02, 8.93866271e-02, 8.72276649e-02],
       ...,
       [1.86860001e+02, 1.84399994e+02, 1.88039993e+02, ...,
        1.92419998e+02, 1.94169998e+02, 1.94500000e+02],
       [1.85850006e+02, 1.86860001e+02, 1.84399994e+02, ...,
        1.91729996e+02, 1.92419998e+02, 1.94169998e+02],
       [1.87679993e+02, 1.85850006e+02, 1.86860001e+02, ...,
        1.88039993e+02, 1.91729996e+02, 1.92419998e+02]])

In [23]:
# Need to scale our data first
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(-1,1))
shift_numpy = scaler.fit_transform(shift_numpy)

shift_numpy

array([[-0.99935033, -0.99939829, -0.99945497, ..., -0.99950293,
        -0.99943317, -0.99938085],
       [-0.99930672, -0.99935033, -0.99939829, ..., -0.99948113,
        -0.99950293, -0.99943317],
       [-0.99925004, -0.99930672, -0.99935033, ..., -0.99945497,
        -0.99948113, -0.99950293],
       ...,
       [ 0.88640494,  0.86156542,  0.89831973, ...,  0.94254612,
         0.96021646,  0.9635486 ],
       [ 0.87620669,  0.88640494,  0.86156542, ...,  0.93557893,
         0.94254612,  0.96021646],
       [ 0.89468468,  0.87620669,  0.88640494, ...,  0.89831973,
         0.93557893,  0.94254612]])

In [25]:
x = shift_numpy[:, 1:]
y = shift_numpy[:, 0]

x.shape, y.shape

((10871, 6), (10871,))

In [26]:
# reverses the x --> the array will be D-7, D-6, D-5, D-4, D-3, D-2, D-1
x = dc(np.flip(x, axis=1))
x

array([[-0.99938085, -0.99943317, -0.99950293, -0.99948113, -0.99945497,
        -0.99939829],
       [-0.99943317, -0.99950293, -0.99948113, -0.99945497, -0.99939829,
        -0.99935033],
       [-0.99950293, -0.99948113, -0.99945497, -0.99939829, -0.99935033,
        -0.99930672],
       ...,
       [ 0.9635486 ,  0.96021646,  0.94254612,  0.93557893,  0.89831973,
         0.86156542],
       [ 0.96021646,  0.94254612,  0.93557893,  0.89831973,  0.86156542,
         0.88640494],
       [ 0.94254612,  0.93557893,  0.89831973,  0.86156542,  0.88640494,
         0.87620669]])